<a href="https://colab.research.google.com/github/hangatzu2017/RAG/blob/main/Mixtral_langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**NB:** Per eseguire questo notebook è necessario possedere un account a pagamento di colab o un jupyter on premise.

Installazione delle librerie necessarie e Download del modello

In [ ]:
!pip install -U -qq transformers accelerate exllamav2 langchain
!apt-get update && apt-get install git-lfs
!git lfs install

In [ ]:
# Download del modello Mixtral 8x7B Instruct (Quantizzato 2bit)
!wget https://huggingface.co/TheBloke/Mixtral-8x7B-Instruct-v0.1-GGUF/resolve/main/mixtral-8x7b-instruct-v0.1.Q2_K.gguf

Selezione del tipo di accellerazione da utilizzare con LLama-cpp-python ed installazione della stessa.

In [ ]:
# Base ctransformers senza accellerazione GPU
#!pip install llama-cpp-python

# Con accellerazione NVidia CUDA ( Questo fa per noi :) )
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python

# Oppure con accellerazione OpenBLAS
#!CMAKE_ARGS="-DLLAMA_BLAS=ON -DLLAMA_BLAS_VENDOR=OpenBLAS" pip install llama-cpp-python

# O ancora con accellerazione CLBLast
#!CMAKE_ARGS="-DLLAMA_CLBLAST=on" pip install llama-cpp-python

# Oppure ancora con accellerazione AMD ROCm GPU (solo per sistemi Linux)
#!CMAKE_ARGS="-DLLAMA_HIPBLAS=on" pip install llama-cpp-python

# Infine con accellerazione Metal GPU per soli sistemi macOS
#!CMAKE_ARGS="-DLLAMA_METAL=on" pip install llama-cpp-python

In [ ]:
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.llms import LlamaCpp

# Parametri di configurazione del modello
n_gpu_layers = 32  # Metal impostato ad 1 è sufficiente.
n_batch = 512  # Dovrebbe essere compreso tra 1 e n_ctx, tieni presente la quantità di RAM del tuo Silicon Chip Apple.
n_ctx=8192 # Il context varia a seconda del modello solitamente il valore max è dichiarato nella scheda di presentazionedel modello stesso.
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

# Definizione del modello
chat_model = LlamaCpp(
    model_path="/content/mixtral-8x7b-instruct-v0.1.Q2_K.gguf",
    n_gpu_layers=n_gpu_layers,
    n_batch=n_batch,
    n_ctx=n_ctx,
    f16_kv=True,  # NB: DEVE essere impostato a True, altrimenti si verificheranno problemi dopo un paio di chiamate.
    callback_manager=callback_manager,
    verbose=True, # Questo è parlante.
)

In [ ]:
import requests

# Definizione della funzione per la lettura della temperatura nella località specificata
def get_weather_data(coordinates):
    """
    Recupera i dati meteo dall'API Open-Meteo per la latitudine e la longitudine specificate.

    Argomenti:
    coordinate (tupla): Latitudine e longitudine della località.

    Restituisce:
    float: Temperatura attuale della località.
    """
    latitude, longitude = coordinates
    base_url = "https://api.open-meteo.com/v1/forecast"
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "current": "temperature_2m,wind_speed_10m",
        "hourly": "temperature_2m,relative_humidity_2m,wind_speed_10m",
    }

    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        return f"""Temperatura(C): {response.json()["current"]["temperature_2m"]}"""
    else:
        return {
            "error": "Impossibile recuperare i dati, status code: {}".format(
                response.status_code
            )
        }


# Definizione della funzione per la lettura
def get_coordinates_from_city(city_name):
    """
    Recupera la latitudine e la longitudine per il nome della città specificata utilizzando l'API di geocodifica di Maps.co.

    Argomenti:
    city_name (str): Nome della città.

    Restituisce:
    tupla: Latitudine e longitudine della città.
    """
    base_url = "https://geocode.maps.co/search"
    params = {"q": city_name}

    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        data = response.json()
        if data:
            # Supponiamo che il primo risultato sia il più rilevante
            return data[0]["lat"], data[0]["lon"]
        else:
            return {"error": "Non sono stati trovati dati per il nome della città specificata."}
    else:
        return {
            "error": "Impossibile recuperare i dati, status code: {}".format(
                response.status_code
            )
        }


In [ ]:
# Definizione del prompt del modello

prompt_template = \
'''
Function:
def get_weather_data(coordinates):
    """
    Fetches weather data from the Open-Meteo API for the given latitude and longitude.

    Args:
    coordinates (tuple): The latitude of the location.

    Returns:
    float: The current temperature in the coordinates you've asked for
    """

Function:
def get_coordinates_from_city(city_name):
    """
    Fetches the latitude and longitude of a given city name using the Maps.co Geocoding API.

    Args:
    city_name (str): The name of the city.

    Returns:
    tuple: The latitude and longitude of the city.
    """

User Query: {query}<human_end>

'''


In [ ]:
from langchain.schema.messages import HumanMessage, SystemMessage

# La nostra Domanda
query = "Com'è il tempo a Roma in questo momento?"
messages = [
    HumanMessage(content=prompt_template.format(query=query))
]

result = chat_model.invoke(messages)

# Stampa del risultato (risposta)
print(result)

# CHAIN

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.chat import (
    HumanMessagePromptTemplate,
)
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda


def call_function(text: str):
    return eval(text)


prompt = ChatPromptTemplate.from_messages(
    [
        HumanMessagePromptTemplate.from_template(prompt_template),
    ]
)

chain = (
    {"query": RunnablePassthrough()}
    | prompt
    | chat_model
    | StrOutputParser()
)

In [ ]:
result = chain.invoke("Quali sono la latitudine e la longitudine di Firenze?")
print(result)